## Initialization

In [ ]:
!git clone https://git.ecker.tech/mrq/ai-voice-cloning/
%cd ai-voice-cloning
!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install -r ./requirements.txt

In [ ]:
# colab requires the runtime to restart before use
exit()

## Running

In [ ]:
%cd ai-voice-cloning
import src.webui as mrq
import sys
sys.argv = [""]

mrq.args = mrq.setup_args()
mrq.webui = mrq.setup_gradio()
mrq.webui.launch(share=True, prevent_thread_lock=True, height=1000)
mrq.tts = mrq.setup_tortoise()
mrq.webui.block_thread()

## Exporting

In [ ]:
!apt install -y p7zip-full
from datetime import datetime
timestamp = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
!mkdir -p "../{timestamp}"
!mv ./results/* "../{timestamp}/."
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "../{timestamp}.7z" "../{timestamp}/"
!ls ~/
!echo "Finished zipping, archive is available at {timestamp}.7z"